In [3]:
import os

In [4]:
%pwd

'/Users/yagito/Desktop/Melenoma-Type-Classification/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/Users/yagito/Desktop/Melenoma-Type-Classification'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from melenomaTypeClassifier.constants import *
from melenomaTypeClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        
        return data_ingestion_config
        

In [10]:
import os 
import gdown
import zipfile
from cancerDetection import logger
from cancerDetection.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            url = self.config.source_URL
            output = self.config.local_data_file
            _, filename = os.path.split(output)
            gdown.download(url, output, quiet=False)
            
            logger.info(f"{filename} downloaded!")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)            

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-16 23:33:57,550: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-16 23:33:57,551: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-16 23:33:57,552: INFO: common: created directory at: artifacts]
[2023-11-16 23:33:57,552: INFO: common: created directory at: artifacts/data_ingestion]


Downloading...
From (uriginal): https://drive.google.com/uc?id=17Khd1kwNctr0UPjl9JWqnBpSpYZJ_nQH
From (redirected): https://drive.google.com/uc?id=17Khd1kwNctr0UPjl9JWqnBpSpYZJ_nQH&confirm=t&uuid=dc420a97-3931-47fc-b7d3-846d6db9384b
To: /Users/yagito/Desktop/Melenoma-Type-Classification/artifacts/data_ingestion/data.zip
100%|██████████| 172M/172M [01:17<00:00, 2.20MB/s] 

[2023-11-16 23:35:16,880: INFO: 4172583660: data.zip downloaded!]
